In [1]:
import numpy as np
import pandas as pd
import scipy.stats as stats
from mll_calc.mll_pred import format_XY, ratios, mll_testset

In [2]:
unc = 0.05
lbls = ['ReactorType', 'CoolingTime', 'Enrichment', 'Burnup', 'OrigenReactor']
tamu_list = ['cs137/cs133', 'cs134/cs137', 'cs135/cs137', 'ba136/ba138',
             'sm150/sm149', 'sm152/sm149', 'eu154/eu153', 'pu240/pu239',
             'pu241/pu239', 'pu242/pu239']

In [3]:
train_db = '~/sims_n_results/sfcompo_compare_aug2020/van2ohi2_not-scaled_15nuc_5.pkl'
test_db = '~/sfcompo/format_clean/sfcompo_formatted.pkl'
XY = format_XY(train_db)
test = pd.read_pickle(test_db)

ratio_list = tamu_list
XY = ratios(XY, ratio_list, lbls)
test = ratios(test, ratio_list, lbls)

## Test sample from SFCOMPO that was simulated in ORIGEN

In [4]:
test = test.loc[test.index == 'OHI-2|17G|C5|89G03']
test

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
OHI-2|17G|C5|89G03,0.0,0.022164,0.0,0.0,0.0,0.0,0.0,0.473684,0.281307,0.137568,pwr,0.0,1.6874,28717.0,Ohi-2_PWR


## OHI-2|17G|C5|89G0 Simulation Parameters:
* PWR: Ohi-2_PWR, so origen equivalent is W(17x17)
* Enrichment: 1.6874
* Burnup: 28717.0 MW*d/tHM
* Cooling time: 1600-2000 days
* U weight: 87 MT (Did 1 MTU in sim)
* Thermal power: 3423 MW for cycles 5 & 6
* Avg power for ORIGEN-ARP is 39 MW/MTHM for sim

In [5]:
XY[6:12]

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor
6,1.117704,0.112794,0.173977,0.012337,192.570281,73.025435,0.226429,0.421967,0.347571,0.211664,pwr,0.0,1.69,28717.0,w17x17
7,0.996161,0.028709,0.193256,0.012786,93.106796,35.307443,0.157377,0.413108,0.273350,0.205983,pwr,1600.0,1.69,28717.0,w17x17
8,0.990403,0.026337,0.194380,0.012786,93.106796,35.307443,0.153934,0.413108,0.269925,0.205983,pwr,1700.0,1.69,28717.0,w17x17
9,0.983685,0.024185,0.195707,0.012786,93.106796,35.307443,0.150574,0.413336,0.266271,0.205983,pwr,1800.0,1.69,28717.0,w17x17
10,0.977927,0.022198,0.196860,0.012786,93.106796,35.307443,0.147295,0.413336,0.262845,0.205983,pwr,1900.0,1.69,28717.0,w17x17
11,0.971209,0.020385,0.198221,0.012786,93.106796,35.307443,0.144098,0.413565,0.259420,0.205983,pwr,2000.0,1.69,28717.0,w17x17


## First, try to find best prediction via MLL

In [6]:
pred_df = mll_testset(XY[6:12].copy(), test, True, unc, lbls)

In [7]:
pd.set_option("display.max_rows", None, "display.max_columns", None)
pred_df

,sim_idx,ReactorType,CoolingTime,Enrichment,Burnup,OrigenReactor,pred_idx,pred_ReactorType,pred_CoolingTime,pred_Enrichment,pred_Burnup,pred_OrigenReactor,MaxLogLL,MaxLLUnc,2ndMaxLogLL,2ndMaxLLUnc,CDF_LogLL_0.9998,CDF_LLUnc_0.9998,CDF_LogLL_0.9988,CDF_LLUnc_0.9988,CDF_LogLL_0.95,CDF_LLUnc_0.95,CDF_LogLL_0.9,CDF_LLUnc_0.9,CDF_LogLL_0.5,CDF_LLUnc_0.5,CDF_LogLL_0.1,CDF_LLUnc_0.1,CDF_LogLL_0.01,CDF_LLUnc_0.01
0,OHI-2|17G|C5|89G03,pwr,0.0,1.6874,28717.0,Ohi-2_PWR,10,pwr,1900.0,1.69,28717.0,w17x17,-11.399112,9.368498,-12.544924,9.558828,-11.400258,19.371377,-11.405987,19.328698,-11.685565,17.245935,-11.972018,15.111957,-14.623191,9.962407,-85.625055,9.463663,-143.510459,9.378014


## Second, dig in and look at ratio diffs

In [8]:
def logpdf_calc(sim_row, test_sample, unc):
    y_sim = sim_row[test_sample>0].values.tolist()
    std = sim_row.multiply(unc)[test_sample>0].values.tolist()
    y_mes = test_sample[test_sample>0].values.tolist()
    logpdf_list = stats.norm.logpdf(y_sim, loc=y_mes, scale=std)
    return logpdf_list

In [9]:
test_sample = test.drop(columns=lbls).squeeze()
test_answer = test[lbls].squeeze()
sim_row = XY.loc[10].drop(lbls).squeeze()
logpdf = logpdf_calc(sim_row, test_sample, unc)
logpdf = pd.Series(logpdf, index=sim_row[test_sample>0].index, name='LogPDF')

In [10]:
df = pd.DataFrame([test_sample, sim_row])
df.loc['Abs Diff'] = np.abs(df.loc[10] - df.loc['OHI-2|17G|C5|89G03'])
df.loc['% Diff'] = df.loc['Abs Diff'] * 100 / df.loc['OHI-2|17G|C5|89G03']

In [11]:
df.loc['LogPDF'] = logpdf

In [12]:
#5 honed in cools
df

,cs137/cs133,cs134/cs137,cs135/cs137,ba136/ba138,sm150/sm149,sm152/sm149,eu154/eu153,pu240/pu239,pu241/pu239,pu242/pu239
OHI-2|17G|C5|89G03,0.000000,0.022164,0.00000,0.000000,0.000000,0.000000,0.000000,0.473684,0.281307,0.137568
10,0.977927,0.022198,0.19686,0.012786,93.106796,35.307443,0.147295,0.413336,0.262845,0.205983
Abs Diff,0.977927,0.000034,0.19686,0.012786,93.106796,35.307443,0.147295,0.060348,0.018461,0.068415
% Diff,inf,0.152594,inf,inf,inf,inf,inf,12.740098,6.562700,49.731779
LogPDF,NaN,5.884072,NaN,NaN,NaN,NaN,NaN,-1.303015,2.426352,-18.406522
